In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-229914
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-229914


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = 'vm-proj'

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({"C":  uniform(0.001, 1), "max_iter": choice(25, 50, 100, 150, 200, 250)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig('.', 'train.py', compute_target=compute_target, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_97992486-5abd-4a44-8872-c97356b9fd0b
Web View: https://ml.azure.com/runs/HD_97992486-5abd-4a44-8872-c97356b9fd0b?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-229914/workspaces/quick-starts-ws-229914&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-04-06T10:03:55.189523][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-04-06T10:03:55.6126057Z][SCHEDULER][INFO]Scheduling job, id='HD_97992486-5abd-4a44-8872-c97356b9fd0b_0' 
[2023-04-06T10:03:55.8927303Z][SCHEDULER][INFO]Scheduling job, id='HD_97992486-5abd-4a44-8872-c97356b9fd0b_2' 
[2023-04-06T10:03:55.922329][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-04-06T10:03:56.0141009Z][SCHEDULER][INFO]Scheduling job, id='HD_97992486-5abd-4a44-8872-c97356b9fd0b_3' 
[2023-04-06T10:03:56.3838740Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_97992486-5abd-4a44-8

{'runId': 'HD_97992486-5abd-4a44-8872-c97356b9fd0b',
 'target': 'vm-proj',
 'status': 'Completed',
 'startTimeUtc': '2023-04-06T10:03:53.937386Z',
 'endTimeUtc': '2023-04-06T10:24:17.268972Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e1a7e586-2413-44b4-bf6a-587381bc55b3',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': 'infinite_space_size',
  'score': '0.9089529590288316',
  'best_child_run_id': 'HD_97992486-5abd-4a44-8872-c97356b9fd0b_5',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_97992486-5abd-4a44-8872-c97356b9fd0b_5'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues'

In [13]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_metrics())


{'Regularization Strength:': 0.5346286481698722, 'Max iterations:': 100, 'Accuracy': 0.9089529590288316}


In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(
    path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv", 
)

In [18]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.

x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_with_y = x_train.join(y_train)

In [22]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=x_train_with_y,
    label_column_name='y',
    n_cross_validations=5)

In [23]:
# Submit your automl run

run = exp.submit(config=automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_a68feaf5-bd0e-4111-993d-1777c051d0d7,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one cl

2023/04/06 11:57:54 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-04-06:12:09:42,168 INFO     [explanation_client.py:334] Using default datastore for uploads


In [31]:
# Retrieve and save your best automl model.

best_run = run.get_best_child()
best_run.get_metrics()


{'matthews_correlation': 0.5412961186437167,
 'f1_score_macro': 0.7670995162347797,
 'recall_score_macro': 0.7406012136498239,
 'f1_score_weighted': 0.912100766040307,
 'recall_score_weighted': 0.9169954476479514,
 'norm_macro_recall': 0.4812024272996478,
 'precision_score_micro': 0.9169954476479514,
 'average_precision_score_macro': 0.827440232597976,
 'precision_score_weighted': 0.9100375356464985,
 'f1_score_micro': 0.9169954476479514,
 'AUC_weighted': 0.9489271366808081,
 'precision_score_macro': 0.8047303319505092,
 'weighted_accuracy': 0.9607975230783415,
 'AUC_micro': 0.981153274032251,
 'log_loss': 0.1788800717837276,
 'average_precision_score_micro': 0.9819253148783179,
 'average_precision_score_weighted': 0.9562147179413382,
 'AUC_macro': 0.948927136680808,
 'balanced_accuracy': 0.7406012136498239,
 'accuracy': 0.9169954476479514,
 'recall_score_micro': 0.9169954476479514,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_a68feaf5-bd0e-4111-993d-1777c051d0d7_13/a

In [35]:
model_name = best_run.properties["model_name"]

best_run.download_file('outputs/model.pkl')

In [40]:
compute_target.delete()